In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Nov 21 17:05:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [3]:
from google.colab import drive
drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("./gdrive/MyDrive/ML DL datasets"))

['fakeimage_detection_2.zip', 'deep_fake_detec1', 'deep_fake_detec2', 'deep_fake_detec3', 'fakeimage_detection_3.zip', 'fake_image_detection_4.zip']


In [ ]:
# unzp archive
!unzip "./gdrive/MyDrive/ML DL datasets/fake_image_detection_4.zip"

In [6]:
training_path = './fake_image_detection_4/train'
test_path = './fake_image_detection_4/test'
validation_path = './fake_image_detection_4/validation'
!ls

fake_image_detection_4	gdrive	__MACOSX  sample_data


In [7]:
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SeparableConv2D as Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

print(tf. __version__) 

2.7.0


In [8]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [9]:
# import keras
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                  #  rotation_range = 10,
                                  #  width_shift_range = 0.1,
                                  #  height_shift_range = 0.1,
                                   horizontal_flip = True)
valid_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
batch_size = 64
input_size=128
training_set = train_datagen.flow_from_directory(training_path,
                                                 target_size = (input_size, input_size),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')

Found 107190 images belonging to 2 classes.


In [11]:
validation_set = valid_datagen.flow_from_directory(validation_path,
                                                 target_size = (input_size, input_size),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')

Found 6699 images belonging to 2 classes.


In [13]:
# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2, verbose=2, baseline=0.801, min_delta=0.001)
checkpoint_filepath = '/content/gdrive/MyDrive/dl_models/checkpoint'
custom_callbacks = [
    EarlyStopping(
        monitor = 'val_loss',
        mode = 'min',
        patience = 5,
        verbose = 1
    ),
    ModelCheckpoint(
        filepath = os.path.join(checkpoint_filepath, 'best_model_EB0_1.h5'),
        monitor = 'val_loss',
        mode = 'min',
        verbose = 1,
        save_best_only = True
    )
]

In [16]:
# efficientnet transfer learning
from efficientnet.tfkeras import EfficientNetB0
efficient_net = EfficientNetB0(
    weights = 'imagenet',
    input_shape = (input_size, input_size, 3),
    include_top = False,
    pooling = 'max'
)

classifier = Sequential()
classifier.add(efficient_net)
classifier.add(Dense(units = 300, activation = 'relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 300, activation = 'relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001)))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 2, activation = 'softmax'))
classifier.summary()
classifier.compile(optimizer = optimizers.Adam(learning_rate=0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

16818176/16804768 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnet-b0 (Functional  (None, 1280)             4049564   
 )                                                               
                                                                 
 dense (Dense)               (None, 300)               384300    
                                                                 
 dropout (Dropout)           (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 300)               90300     
                                                                 
 dropout_1 (Dropout)         (None, 300)               0         
                                                                 
 dense_2 (Dense)             (None, 2)                 60

In [17]:
# 1693 or 3386
history = classifier.fit(
                        training_set,
                        steps_per_epoch=len(training_set),
                        epochs=10,
                        verbose=1,
                        validation_data=validation_set,
                        callbacks = custom_callbacks
                         )

Epoch 1/10
1675/1675 [==============================] - ETA: 0s - loss: 1.4040 - accuracy: 0.9111
Epoch 00001: val_loss improved from inf to 0.28739, saving model to /content/gdrive/MyDrive/dl_models/checkpoint/best_model_EB0_1.h5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



1675/1675 [==============================] - 1029s 598ms/step - loss: 1.4040 - accuracy: 0.9111 - val_loss: 0.2874 - val_accuracy: 0.8986
Epoch 2/10
1675/1675 [==============================] - ETA: 0s - loss: 0.2228 - accuracy: 0.9399
Epoch 00002: val_loss did not improve from 0.28739
1675/1675 [==============================] - 939s 561ms/step - loss: 0.2228 - accuracy: 0.9399 - val_loss: 0.3044 - val_accuracy: 0.9139
Epoch 3/10
1675/1675 [==============================] - ETA: 0s - loss: 0.2004 - accuracy: 0.9501
Epoch 00003: val_loss improved from 0.28739 to 0.20223, saving model to /content/gdrive/MyDrive/dl_models/checkpoint/best_model_EB0_1.h5
1675/1675 [==============================] - 929s 554ms/step - loss: 0.2004 - accuracy: 0.9501 - val_loss: 0.2022 - val_accuracy: 0.9433
Epoch 4/10
1675/1675 [==============================] - ETA: 0s - loss: 0.1842 - accuracy: 0.9564
Epoch 00004: val_lo

In [ ]:
save_path = '/content/gdrive/MyDrive/dl_models/'
model_name = 'categorical_fake_image_classifierEfB80.h5'
classifier.save(save_path + model_name)

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
!ls  /content/gdrive/MyDrive/dl_models

fake_image_classifierEfB80.h5


In [18]:
from keras.models import load_model
best_model = '/content/gdrive/MyDrive/dl_models/checkpoint/best_model_EB0_1.h5'
model = load_model(best_model)
# classifier = model

**Prediction section**

In [87]:
def predict_images(folder, image_name):
    #Making Single new prediction
    import numpy as np
    #from keras.preprocessing.image import image
    import keras.preprocessing.image as image
    test_image = image.load_img(f'./fake_image_detection_4/test/{folder}/{image_name}', target_size = (input_size, input_size))
    test_image = image.img_to_array(test_image)
    test_image = test_image /255.
    #use this function to change the image from 2dimension(64, 64) to 3dimension(64, 64, 3)
    test_image = np.expand_dims(test_image, axis=0)
    #add one more dimension before predicting because the predict method expects a batch
    images = np.vstack([test_image])
    result = model.predict(images, batch_size=10)

    prediction = ''
    if np.argmax(result) == 0:
      prediction = 'fake'
    elif np.argmax(result) == 1:
      prediction = 'real'
    return prediction

In [88]:

import os
fake_images = os.listdir("./fake_image_detection_4/test/fake")
real_images = os.listdir("./fake_image_detection_4/test/real")

res_fake = []
res_real = []

for image_name in fake_images:
    res_fake.append(predict_images('fake', image_name))
for image_name in real_images:
    res_real.append(predict_images('real', image_name))

In [89]:
from collections import Counter
results = (Counter(res_fake), Counter(res_real))
results

(Counter({'fake': 9949, 'real': 100}), Counter({'fake': 552, 'real': 9497}))